In [1]:
# mount drive for access to the files
from google.colab import drive

drive.mount("/content/drive")

# all the drive the files are present in "/content/drive/My Drive"
!ls "/content/drive/My Drive/Beuth Uni/Master Thesis"

import sys
sys.path.append('/content/drive/My Drive/Beuth Uni/Master Thesis/jenga')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Data
'Data Quality in ML Production Systems.pdf'
'Datawig: Missing Value Imputation for Tables.pdf'
 Declaration
 Images
 jenga
 jenga.pdf
 MICE_Multivariate_Imputation_by_Chained_Equations_.pdf


In [2]:
!pip install openml
!pip install pyod
!pip install datawig ##

In [2]:
import random
import numpy as np
import pandas as pd

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from jenga.basis import Dataset
from jenga.corruptions.generic import MissingValues, SwappedValues
from jenga.corruptions.numerical import Scaling, GaussianNoise
from jenga.cleaning.ppp import PipelinePerformancePrediction
from jenga.cleaning.outlier_detection import NoOutlierDetection, PyODKNN, PyODIsolationForest
from jenga.cleaning.imputation import NoImputation, MeanModeImputation, DatawigImputation
from jenga.cleaning.clean import Clean

In [3]:
seed = 10

In [4]:
def run_experiment(dataset_name, learner, param_grid, corruptions, fraction, cleaners):
    
    ## dataset
    dataset = Dataset(seed, dataset_name)
    
    all_data = dataset.all_data
    attribute_names = dataset.attribute_names
    attribute_types = dataset.attribute_types
    
    ## categorical and numerical features
    categorical_columns = dataset.categorical_columns
    numerical_columns = dataset.numerical_columns
    print(f"Found {len(categorical_columns)} categorical and {len(numerical_columns)} numeric features \n")
    
    ## train and test data
    df_train, lab_train, df_test, lab_test = dataset.get_train_test_data()
    
    
    ## pipeline performance prediction (ppp)
    ppp = PipelinePerformancePrediction(seed, df_train, lab_train, df_test, lab_test, categorical_columns, numerical_columns, learner, param_grid)
    ppp_model = ppp.fit_ppp(df_train)
    
    ## generate corrpted data
#     for _ in range(100):
    df_corrupted, perturbations, cols_perturbed, summary_col_corrupt = ppp.get_corrupted(df_test, corruptions, fraction)
    
    ## cleaning
    clean = Clean(df_train, df_corrupted, categorical_columns, numerical_columns, ppp, ppp_model, cleaners)
    df_cleaned, corrupted_score_ppp, best_cleaning_score, cleaner_scores_ppp, summary_cleaners = clean(df_train, df_corrupted)
    
    ## results
    result = {
        'ppp_score_model': ppp.predict_score_ppp(ppp_model, df_test),
        'ppp_score_corrupted': corrupted_score_ppp,
        'ppp_score_cleaned': best_cleaning_score,
        'ppp_scores_cleaners': cleaner_scores_ppp
    }
#     print('\n'.join([f'{key}:{val}' for key, val in result.items()]))
    
    ## summary
    summary = {
        'dataset': dataset_name,
        'model': learner,
        'corruptions': summary_col_corrupt,
        'cleaners': summary_cleaners,
        'result': result
    }
#     print('\n\n\n\n'.join([f'{key}:{val}' for key, val in summary.items()]))
    
    return summary #summary_col_corrupt, result

In [5]:
datasets = [
    'parkinsons',
    'heart-statlog',
    'credit-g'
]

In [6]:
## model parameters
## models is a dict where key = leaner & value = param_grid
models = {SGDClassifier(loss='log'): {'learner__max_iter': [500, 1000, 5000], 
                                         'learner__penalty': ['l2', 'l1', 'elasticnet'], 
                                         'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
                                        }, 
          RandomForestClassifier():{'learner__n_estimators': [100, 200, 500], 
                                    'learner__max_depth': [5, 10, 15]
                                   }
         }

## make dict of multiple leraners and corresponding param_grids

In [7]:
corruptions = [MissingValues, SwappedValues, Scaling, GaussianNoise]

In [8]:
fractions = [0.25, 0.5, 0.75]

In [9]:
cleaners = [
    (NoOutlierDetection, MeanModeImputation),
    (PyODKNN, NoImputation),
    (PyODKNN, MeanModeImputation),
    (PyODIsolationForest, NoImputation),
    (PyODIsolationForest, MeanModeImputation)
]

In [10]:
ind_results = []

# for _ in range(100):
for dataset in datasets:
    for learner, param_grid in models.items():
        for fraction in fractions:
            ind_results.append(run_experiment(dataset, learner, param_grid, [MissingValues], fraction, [(PyODKNN, MeanModeImputation)]))

Dataset: parkinsons
Found 0 categorical and 22 numeric features 

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    3.3s finished


Generating corrupted training data on 39 rows... 

Applying perturbations... 

MissingValues: {'column': 'V2', 'fraction': 0.25, 'na_value': nan, 'missingness': 'MNAR'}

Applying cleaners... 

PPP score no cleaning: {'roc_auc_acore': 0.8181818181818181, 'classification_report': {'1': {'precision': 0.6666666666666666, 'recall': 0.5454545454545454, 'f1-score': 0.6, 'support': 11}, '2': {'precision': 0.8333333333333334, 'recall': 0.8928571428571429, 'f1-score': 0.8620689655172413, 'support': 28}, 'accuracy': 0.7948717948717948, 'macro avg': {'precision': 0.75, 'recall': 0.7191558441558441, 'f1-score': 0.7310344827586206, 'support': 39}, 'weighted avg': {'precision': 0.7863247863247863, 'recall': 0.7948717948717948, 'f1-score': 0.7881520778072502, 'support': 39}}}
PPP score with cleaning: Cleaner: {'outlier_detection': PyODKNN, 'imputation': MeanModeImputation}: {'roc_auc_acore': 0.724025974025974, 'classification_report': {'1': {'precision': 0.5, 'recall': 0.6363636363636364, 'f1-score': 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.8s finished


Generating corrupted training data on 39 rows... 

Applying perturbations... 

MissingValues: {'column': 'V2', 'fraction': 0.5, 'na_value': nan, 'missingness': 'MNAR'}

Applying cleaners... 

PPP score no cleaning: {'roc_auc_acore': 0.9902597402597402, 'classification_report': {'1': {'precision': 0.8333333333333334, 'recall': 0.9090909090909091, 'f1-score': 0.8695652173913043, 'support': 11}, '2': {'precision': 0.9629629629629629, 'recall': 0.9285714285714286, 'f1-score': 0.9454545454545454, 'support': 28}, 'accuracy': 0.9230769230769231, 'macro avg': {'precision': 0.8981481481481481, 'recall': 0.9188311688311688, 'f1-score': 0.9075098814229249, 'support': 39}, 'weighted avg': {'precision': 0.9264007597340932, 'recall': 0.9230769230769231, 'f1-score': 0.9240498631802979, 'support': 39}}}
PPP score with cleaning: Cleaner: {'outlier_detection': PyODKNN, 'imputation': MeanModeImputation}: {'roc_auc_acore': 0.9123376623376623, 'classification_report': {'1': {'precision': 0.7272727272727273

...

Generating corrupted training data on 200 rows... 

Applying perturbations... 

MissingValues: {'column': 'credit_amount', 'fraction': 0.25, 'na_value': nan, 'missingness': 'MNAR'}

Applying cleaners... 

PPP score no cleaning: {'roc_auc_acore': 0.7732585320243104, 'classification_report': {'bad': {'precision': 0.6842105263157895, 'recall': 0.41935483870967744, 'f1-score': 0.5200000000000001, 'support': 62}, 'good': {'precision': 0.7777777777777778, 'recall': 0.9130434782608695, 'f1-score': 0.84, 'support': 138}, 'accuracy': 0.76, 'macro avg': {'precision': 0.7309941520467836, 'recall': 0.6661991584852734, 'f1-score': 0.68, 'support': 200}, 'weighted avg': {'precision': 0.7487719298245614, 'recall': 0.76, 'f1-score': 0.7408000000000001, 'support': 200}}}
PPP score with cleaning: Cleaner: {'outlier_detection': PyODKNN, 'imputation': MeanModeImputation}: {'roc_auc_acore': 0.7765310892940627, 'classification_report': {'bad': {'precision': 0.696969696969697, 'recall': 0.3709677419354839, '

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   14.0s finished


Generating corrupted training data on 200 rows... 

Applying perturbations... 

MissingValues: {'column': 'credit_amount', 'fraction': 0.5, 'na_value': nan, 'missingness': 'MNAR'}

Applying cleaners... 

PPP score no cleaning: {'roc_auc_acore': 0.793419822346891, 'classification_report': {'bad': {'precision': 0.6944444444444444, 'recall': 0.4032258064516129, 'f1-score': 0.5102040816326531, 'support': 62}, 'good': {'precision': 0.774390243902439, 'recall': 0.9202898550724637, 'f1-score': 0.8410596026490066, 'support': 138}, 'accuracy': 0.76, 'macro avg': {'precision': 0.7344173441734417, 'recall': 0.6617578307620383, 'f1-score': 0.6756318421408298, 'support': 200}, 'weighted avg': {'precision': 0.7496070460704607, 'recall': 0.76, 'f1-score': 0.738494391133937, 'support': 200}}}
PPP score with cleaning: Cleaner: {'outlier_detection': PyODKNN, 'imputation': MeanModeImputation}: {'roc_auc_acore': 0.7736091631603553, 'classification_report': {'bad': {'precision': 0.7, 'recall': 0.3387096774

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   14.1s finished


Generating corrupted training data on 200 rows... 

Applying perturbations... 

MissingValues: {'column': 'credit_amount', 'fraction': 0.75, 'na_value': nan, 'missingness': 'MNAR'}

Applying cleaners... 

PPP score no cleaning: {'roc_auc_acore': 0.7826086956521741, 'classification_report': {'bad': {'precision': 0.7, 'recall': 0.45161290322580644, 'f1-score': 0.5490196078431372, 'support': 62}, 'good': {'precision': 0.7875, 'recall': 0.9130434782608695, 'f1-score': 0.8456375838926175, 'support': 138}, 'accuracy': 0.77, 'macro avg': {'precision': 0.7437499999999999, 'recall': 0.682328190743338, 'f1-score': 0.6973285958678773, 'support': 200}, 'weighted avg': {'precision': 0.7603749999999999, 'recall': 0.77, 'f1-score': 0.7536860113172785, 'support': 200}}}
PPP score with cleaning: Cleaner: {'outlier_detection': PyODKNN, 'imputation': MeanModeImputation}: {'roc_auc_acore': 0.7731416549789621, 'classification_report': {'bad': {'precision': 0.696969696969697, 'recall': 0.3709677419354839, '

In [11]:
ind_results[1]

{'cleaners': [{'Imputation method': MeanModeImputation,
   'Outlier detection method': PyODKNN,
   'PPP score with cleaning': {'classification_report': {'1': {'f1-score': 0.7272727272727273,
      'precision': 0.7272727272727273,
      'recall': 0.7272727272727273,
      'support': 11},
     '2': {'f1-score': 0.8928571428571429,
      'precision': 0.8928571428571429,
      'recall': 0.8928571428571429,
      'support': 28},
     'accuracy': 0.8461538461538461,
     'macro avg': {'f1-score': 0.8100649350649352,
      'precision': 0.8100649350649352,
      'recall': 0.8100649350649352,
      'support': 39},
     'weighted avg': {'f1-score': 0.8461538461538461,
      'precision': 0.8461538461538461,
      'recall': 0.8461538461538461,
      'support': 39}},
    'roc_auc_acore': 0.9123376623376623}}],
 'corruptions': defaultdict(list,
             {('V2',): [MissingValues: {'column': 'V2', 'fraction': 0.5, 'na_value': nan, 'missingness': 'MNAR'}]}),
 'dataset': 'parkinsons',
 'model': SGDC

In [12]:
results = []

# for _ in range(100):
for dataset in datasets:
    for learner, param_grid in models.items():
        for fraction in fractions:
            results.append(run_experiment(dataset, learner, param_grid, corruptions, fraction, cleaners))

Dataset: parkinsons
Found 0 categorical and 22 numeric features 

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.9s finished
2020-07-22 13:43:28,390 [INFO]  NumExpr defaulting to 2 threads.


Generating corrupted training data on 39 rows... 

Can't apply the SwappedValues corruption because there are no categorical columns. 


Applying perturbations... 

MissingValues: {'column': 'V2', 'fraction': 0.25, 'na_value': nan, 'missingness': 'MNAR'}
Scaling: {'column': 'V14', 'fraction': 0.25}
GaussianNoise: {'column': 'V16', 'fraction': 0.25}

Applying cleaners... 

PPP score no cleaning: {'roc_auc_acore': 0.7597402597402597, 'classification_report': {'1': {'precision': 0.4782608695652174, 'recall': 1.0, 'f1-score': 0.6470588235294118, 'support': 11}, '2': {'precision': 1.0, 'recall': 0.5714285714285714, 'f1-score': 0.7272727272727273, 'support': 28}, 'accuracy': 0.6923076923076923, 'macro avg': {'precision': 0.7391304347826086, 'recall': 0.7857142857142857, 'f1-score': 0.6871657754010696, 'support': 39}, 'weighted avg': {'precision': 0.8528428093645485, 'recall': 0.6923076923076923, 'f1-score': 0.704648292883587, 'support': 39}}}
PPP score with cleaning: Cleaner: {'outlier_detec

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.8s finished


Generating corrupted training data on 39 rows... 

Applying perturbations... 

MissingValues: {'column': 'V2', 'fraction': 0.5, 'na_value': nan, 'missingness': 'MNAR'}
Scaling: {'column': 'V14', 'fraction': 0.5}
GaussianNoise: {'column': 'V16', 'fraction': 0.5}

Applying cleaners... 

PPP score no cleaning: {'roc_auc_acore': 0.9805194805194806, 'classification_report': {'1': {'precision': 1.0, 'recall': 0.6363636363636364, 'f1-score': 0.7777777777777778, 'support': 11}, '2': {'precision': 0.875, 'recall': 1.0, 'f1-score': 0.9333333333333333, 'support': 28}, 'accuracy': 0.8974358974358975, 'macro avg': {'precision': 0.9375, 'recall': 0.8181818181818181, 'f1-score': 0.8555555555555556, 'support': 39}, 'weighted avg': {'precision': 0.9102564102564102, 'recall': 0.8974358974358975, 'f1-score': 0.8894586894586894, 'support': 39}}}
PPP score with cleaning: Cleaner: {'outlier_detection': NoOutlierDetection, 'imputation': MeanModeImputation}: {'roc_auc_acore': 0.9805194805194806, 'classificati

...

Generating corrupted training data on 200 rows... 

Applying perturbations... 

MissingValues: {'column': 'credit_amount', 'fraction': 0.25, 'na_value': nan, 'missingness': 'MNAR'}
Scaling: {'column': 'residence_since', 'fraction': 0.25}
GaussianNoise: {'column': 'residence_since', 'fraction': 0.25}

Applying cleaners... 

PPP score no cleaning: {'roc_auc_acore': 0.774310425432445, 'classification_report': {'bad': {'precision': 0.6666666666666666, 'recall': 0.3870967741935484, 'f1-score': 0.4897959183673469, 'support': 62}, 'good': {'precision': 0.7682926829268293, 'recall': 0.9130434782608695, 'f1-score': 0.8344370860927153, 'support': 138}, 'accuracy': 0.75, 'macro avg': {'precision': 0.717479674796748, 'recall': 0.6500701262272089, 'f1-score': 0.662116502230031, 'support': 200}, 'weighted avg': {'precision': 0.7367886178861789, 'recall': 0.75, 'f1-score': 0.7275983240978511, 'support': 200}}}
PPP score with cleaning: Cleaner: {'outlier_detection': NoOutlierDetection, 'imputation': M

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   14.3s finished


Generating corrupted training data on 200 rows... 

Applying perturbations... 

MissingValues: {'column': 'credit_amount', 'fraction': 0.5, 'na_value': nan, 'missingness': 'MNAR'}
Scaling: {'column': 'residence_since', 'fraction': 0.5}
GaussianNoise: {'column': 'residence_since', 'fraction': 0.5}

Applying cleaners... 

PPP score no cleaning: {'roc_auc_acore': 0.7705703599812997, 'classification_report': {'bad': {'precision': 0.6388888888888888, 'recall': 0.3709677419354839, 'f1-score': 0.4693877551020407, 'support': 62}, 'good': {'precision': 0.7621951219512195, 'recall': 0.9057971014492754, 'f1-score': 0.8278145695364238, 'support': 138}, 'accuracy': 0.74, 'macro avg': {'precision': 0.7005420054200542, 'recall': 0.6383824216923797, 'f1-score': 0.6486011623192323, 'support': 200}, 'weighted avg': {'precision': 0.7239701897018971, 'recall': 0.74, 'f1-score': 0.716702257061765, 'support': 200}}}
PPP score with cleaning: Cleaner: {'outlier_detection': NoOutlierDetection, 'imputation': Me

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   14.0s finished


Generating corrupted training data on 200 rows... 

Applying perturbations... 

MissingValues: {'column': 'credit_amount', 'fraction': 0.75, 'na_value': nan, 'missingness': 'MNAR'}
Scaling: {'column': 'residence_since', 'fraction': 0.75}
GaussianNoise: {'column': 'residence_since', 'fraction': 0.75}

Applying cleaners... 

PPP score no cleaning: {'roc_auc_acore': 0.7793361383824217, 'classification_report': {'bad': {'precision': 0.6666666666666666, 'recall': 0.3870967741935484, 'f1-score': 0.4897959183673469, 'support': 62}, 'good': {'precision': 0.7682926829268293, 'recall': 0.9130434782608695, 'f1-score': 0.8344370860927153, 'support': 138}, 'accuracy': 0.75, 'macro avg': {'precision': 0.717479674796748, 'recall': 0.6500701262272089, 'f1-score': 0.662116502230031, 'support': 200}, 'weighted avg': {'precision': 0.7367886178861789, 'recall': 0.75, 'f1-score': 0.7275983240978511, 'support': 200}}}
PPP score with cleaning: Cleaner: {'outlier_detection': NoOutlierDetection, 'imputation': 

In [13]:
results

[{'cleaners': [{'Imputation method': MeanModeImputation,
    'Outlier detection method': NoOutlierDetection,
    'PPP score with cleaning': {'classification_report': {'1': {'f1-score': 0.6470588235294118,
       'precision': 0.4782608695652174,
       'recall': 1.0,
       'support': 11},
      '2': {'f1-score': 0.7272727272727273,
       'precision': 1.0,
       'recall': 0.5714285714285714,
       'support': 28},
      'accuracy': 0.6923076923076923,
      'macro avg': {'f1-score': 0.6871657754010696,
       'precision': 0.7391304347826086,
       'recall': 0.7857142857142857,
       'support': 39},
      'weighted avg': {'f1-score': 0.704648292883587,
       'precision': 0.8528428093645485,
       'recall': 0.6923076923076923,
       'support': 39}},
     'roc_auc_acore': 0.7500000000000001}},
   {'Imputation method': NoImputation,
    'Outlier detection method': PyODKNN,
    'PPP score with cleaning': {'classification_report': {'1': {'f1-score': 0.28571428571428564,
       'precisi